## Import necessary modules
Run this cell before running any other cells

In [ ]:
from uuid import uuid4
uuid4()

In [ ]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import time
import numpy as np

LOG.propagate = False

# Printing and Logging
## Printing
You can use the **print()** function in Python to print messages to the screen. <br>
The message can be a string, or any other object, the object will be converted into a string before it is written to the screen. <br>

## Logging
You could use the logging module that is setup in *utils.py*. <br>
It prints to both your screen (standard output) as well as to log files (*ble.log*) in the *logs* directory. <br>
This is the recommended way to output messages, since the log files can help with debugging. <br>
The logging module also provides different log levels as shown below, each formatted with a different color for increased visibility. <br>

__**NOTE**__: You may notice that the DEBUG message is not printed to the screen but is printed in the log file. This is because the logging level for the screen is set to INFO and for the file is set to DEBUG. You can change the default log levels in *utils.py* (**STREAM_LOG_LEVEL** and **FILE_LOG_LEVEL**). 

## Formatting output
To format your strings, you may use %-formatting, str.format() or f-strings. <br>
The most "pythonic" way would be to use f-strings. [Here](https://realpython.com/python-f-strings/) is a good tutorial on f-strings. <br>

In [ ]:
LOG.debug("debug")
LOG.info("info")
LOG.warning("warning")
LOG.error("error")
LOG.critical("critical")

<hr>

# BLE
## ArtemisBLEController
The class **ArtemisBLEController** (defined in *ble.py*) provides member functions to handle various BLE operations to send and receive data to/from the Artemis board, provided the accompanying Arduino sketch is running on the Artemis board. <br>

<table align="left">
     <tr>
        <th style="text-align: left; font-size: medium">Member Functions</th>
        <th style="text-align: left; font-size: medium">Description</th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">reload_config()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Reload changes made in <em>connection.yaml.</em></span></th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">connect()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Connect to the Artemis board, whose MAC address is specified in <em>connection.yaml</em>.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">disconnect()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Disconnect from the Artemis board.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">is_connected()</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Return a boolean indicating whether your controller is connected to the Artemis board or not.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">send_command(cmd_type, data)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Send the command <strong>cmd_type</strong> (integer) with <strong>data</strong> (string) to the Artemis board.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">receive_float(uuid) <br> receive_string(uuid) <br> receive_int(uuid)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Read the GATT characteristic (specified by its <strong>uuid</strong>) of type float, string or int. <br> The type of the GATT
            characteristic is determined by the classes BLEFloatCharacteristic, BLECStringCharacteristic or
            BLEIntCharacteristic in the Arduino sketch.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">start_notify(uuid, notification_handler)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Activate notifications on the GATT characteristic (specified by its <strong>uuid</strong>). <br> <strong>notification_handler</strong> is a
            function callback which must accept two inputs; the first will be a uuid string object and the second will
            be the bytearray of the characteristic value.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">bytearray_to_float(byte_array) <br> bytearray_to_string(byte_array) <br> bytearray_to_int(byte_array)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Convert the <strong>bytearray</strong> to float, string or int, respectively. <br> You may use these functions inside your
            notification callback function.</span></th>
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">stop_notify(uuid)</span></th>
        <th style="text-align: left"><span style="font-weight: normal">Stop notifications on the GATT characteristic (specified by its <strong>uuid</strong>).</span></th>
    </tr>
</table>

<table align="left">
     <tr>
        <th style="text-align: left; font-size: medium">Member Variables</th>
        <th style="text-align: left; font-size: medium">Description</th style="text-align: left">
    </tr>
    <tr>
        <th style="text-align: left"><span style="color:rgb(201,152,4);font-family:monospace">uuid</span></th>
        <th style="text-align: left"><span style="font-weight: normal">A dictionary that stores the UUIDs of the various characteristics specified in <em>connection.yaml</em>.</span></th>
    </tr>
</table>

## Configuration
- The MAC address, Service UUID and GATT characteristic UUIDs are defined in the file: *connection.yaml*.
- They should match the UUIDs used in the Arduino sketch.
- The artemis board running the base code should display its MAC address in the serial monitor.
- Update the **artemis_address** in *connection.yaml*, accordingly.
- Make sure to call **ble.reload_config()** or **get_ble_controller()** (which internally calls **reload_config()**) after making any changes to your configuration file.

<hr>

In the below cell, we create an **ArtemisBLEController** object using **get_ble_controller()** (defined in *ble.py*), which creates and/or returns a single instance of **ArtemisBLEController**. <br>
<span style="color:rgb(240,50,50)"> __NOTE__: Do not use the class directly to instantiate an object. </span><br>

In [ ]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

## Receive data from the Artemis board

The cell below shows examples of reading different types (as defined in the Arduino sketch) of GATT characteristics.

In [ ]:
# Read a float GATT Charactersistic
f = ble.receive_float(ble.uuid['RX_FLOAT'])
print(f)

In [ ]:
# Read a string GATT Charactersistic
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

## Send a command to the Artemis board
Send the PING command and read the reply string from the string characteristic RX_STRING. <br>
__NOTE__: The **send_command()** essentially sends a string data to the GATT characteristic (TX_CMD_STRING). The GATT characteristic in the Arduino sketch is of type BLECStringCharacteristic.

In [ ]:
ble.send_command(CMD.PING, "")

In [ ]:
s = ble.receive_string(ble.uuid['RX_STRING'])
print(s)

The cell below shows an example of the SEND_TWO_INTS command. <br> The two values in the **data** are separated by a delimiter "|". <br>
Refer Lab 2 documentation for more information on the command protocol.

In [ ]:
ble.send_command(CMD.SEND_TWO_INTS, "2|-6")

The Artemis board should print the two integers to the serial monitor in the ArduinoIDE. 

## Disconnect

In [ ]:
# Disconnect
ble.disconnect()

# Lab

## Imports

In [ ]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import time
import numpy as np

LOG.propagate = False

## Connect

In [ ]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

## Task 1

In [ ]:
ble.send_command(CMD.ECHO, "QUACK!")

In [ ]:
echo = ble.receive_string(ble.uuid['RX_STRING'])
print(echo)

## Task 2

In [ ]:
ble.send_command(CMD.GET_TIME_MILLIS, "")

In [ ]:
millis = ble.receive_string(ble.uuid['RX_STRING'])
print(millis)

## Task 3

In [ ]:
time = 0

def str_notify_handler(uuid, byte_array):
    global time
    byte_array = ble.bytearray_to_string(byte_array)
    time = byte_array[2:]
    time = int(time)
    print("Time:", time, "ms")

In [ ]:
ble.start_notify(ble.uuid['RX_STRING'], str_notify_handler)

In [ ]:
ble.stop_notify(ble.uuid['RX_STRING'])

## Task 4

In [ ]:
times = []
temps = []

def str_notify_handler(uuid, byte_array):
    lst = []
    global times
    global temps
    byte_array = ble.bytearray_to_string(byte_array)

    while(True):
        try:
            index = byte_array.index('|')
            lst.append(byte_array[:index])
            byte_array = byte_array[index+1:]
        except:
            lst.append(byte_array)
            break

    for i in range(len(lst)):
        if i%2 == 0:
            times.append(lst[i][2:])
        else:
            temps.append(lst[i][2:])

    print("Time(ms)    ", "Temperature(C)")
    for i in range(len(times)):
        print(times[i], "      ", temps[i])

In [ ]:
ble.start_notify(ble.uuid['RX_STRING'], str_notify_handler)

In [ ]:
ble.stop_notify(ble.uuid['RX_STRING'])

In [ ]:
ble.send_command(CMD.GET_TEMP_5s, "")

## Task 5

In [ ]:
lst = []
def str_notify_handler(uuid, byte_array):
    global times
    global temps
    global lst
    data = ble.bytearray_to_string(byte_array)
    
    while(True):
        try:
            index = data.index('|')
            lst.append(data[:index])
            data = data[index+1:]
        except:
            #lst.append(data)
            break

    #print("5s of temperature readings stored in temps list.")

In [ ]:
ble.start_notify(ble.uuid['RX_STRING'], str_notify_handler)

In [ ]:
ble.stop_notify(ble.uuid['RX_STRING'])

In [ ]:
ble.send_command(CMD.GET_TEMP_5s_RAPID, "")

In [ ]:
times = []
temps = []

for i in range(len(lst)):
    if i%2 == 0:
        times.append(lst[i][2:])
    else:
        temps.append(lst[i][2:])

print("Time(ms)    ", "Temperature(C)")
for i in range(len(times)):
    print(times[i], "      ", temps[i])
# print(lst)
# print(times)
# print(temps)
print("Total # of temperature measurements taken:", len(temps))

## Lab 3 Task 9

In [ ]:
import numpy as np
lst = []
means = np.zeros(15)
stdevs = np.zeros(15)
def str_notify_handler(uuid, byte_array):
    global lst
    data = ble.bytearray_to_string(byte_array)
    print(data)
    
    while(True):
        try:
            index = data.index('|')
            lst.append(data[:index])
            data = data[index+1:]
        except:
            break

In [ ]:
ble.start_notify(ble.uuid['RX_STRING'], str_notify_handler)

In [ ]:
ble.stop_notify(ble.uuid['RX_STRING'])

In [ ]:
ble.send_command(CMD.GET_DIST, "5000")

In [ ]:
import statistics as stat
times = []
dist1 = []
dist2 = []

counter = 0
for i in range(len(lst)):
    if counter == 0:
        times.append(int(lst[i][2:]))
    elif counter == 1:
        dist1.append(int(lst[i][3:]))
    elif counter == 2:
        dist2.append(int(lst[i][3:]))
    counter += 1
    counter = counter % 3

print("Time(ms)    ", "Distance 1(cm)    ", "Distance 2(cm)    ")
for i in range(len(times)):
    print(times[i], "      ", dist1[i], "              ", dist2[i])

print("Average distances: ", stat.mean(dist1), "              ", stat.mean(dist2))
print("Standard deviations: ", stat.stdev(dist1), "              ", stat.stdev(dist2))

index = 1 # CHANGE THIS for each distance
means[index-1] = stat.mean(dist2)
stdevs[index-1] = stat.stdev(dist2)
print(means)
print(stdevs)
lst = []

In [ ]:
savemeans = means
savestdevs = stdevs

In [ ]:
real_distances = [2,4,6,8,10,12,14,16,18,20,22,24,26,28,30]
plt.plot(real_distances, means, label = 'Mean sensor reading')
plt.plot(real_distances, stdevs, label = 'Standard deviation')
plt.xlabel('Actual Distance(cm)')
plt.ylabel('Sensor Distance(cm)')
plt.legend()
plt.show()

In [ ]:
pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt
plt.plot(times, dist1)
plt.plot(times, dist2)
plt.xlabel('Time(ms)')
plt.ylabel('Distance(cm)')
plt.show()

### Imports and connect to BLE again for convenience

In [ ]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import time
import numpy as np

LOG.propagate = False

In [ ]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

## Lab 4

### IMU only

In [ ]:
import numpy as np
lst = []
def str_notify_handler(uuid, byte_array):
    global lst
    data = ble.bytearray_to_string(byte_array)
    print(data)
    
    while(True):
        try:
            index = data.index('|')
            lst.append(data[:index])
            data = data[index+1:]
        except:
            break

In [ ]:
ble.start_notify(ble.uuid['RX_STRING'], str_notify_handler)

In [ ]:
ble.stop_notify(ble.uuid['RX_STRING'])

In [ ]:
ble.send_command(CMD.GET_IMU, "5000")

In [ ]:
import statistics as stat
times = []
pitch = []
roll = []
yaw = []

counter = 0
for i in range(len(lst)):
    if counter == 0:
        times.append(float(lst[i][2:]))
    elif counter == 1:
        pitch.append(float(lst[i][2:]))
    elif counter == 2:
        roll.append(float(lst[i][2:]))
    elif counter == 3:
        yaw.append(float(lst[i][2:]))
    counter += 1
    counter = counter % 4

print("Time(ms)      ", "Pitch          ", "Roll            ", "Yaw      ")
for i in range(len(times)):
    print(times[i], "      ", pitch[i], "        ", roll[i], "        ", yaw[i])

### TOF & IMU

In [ ]:
import numpy as np
tof = []
imu = []
def str_notify_handler(uuid, byte_array):
    global tof
    global imu
    data = ble.bytearray_to_string(byte_array)
    print(data)
    
    if (int(data[0]) == 1): #TOF data
        while(True):
            try:
                data = data[1:]
                index = data.index('|')
                tof.append(data[:index])
                data = data[index+1:]
            except:
                break
    elif (int(data[0]) == 2): #IMU data
        while(True):
            try:
                data = data[1:]
                index = data.index('|')
                imu.append(data[:index])
                data = data[index+1:]
            except:
                break
    else:
        print("Data is not classified as TOF or IMU")

In [ ]:
ble.start_notify(ble.uuid['RX_STRING'], str_notify_handler)

In [ ]:
ble.stop_notify(ble.uuid['RX_STRING'])

In [ ]:
ble.send_command(CMD.GET_TOF_IMU, "5000")

In [ ]:
print(imu)

In [ ]:
import statistics as stat
times = []
pitch = []
roll = []
yaw = []

counter = 0
for i in range(len(imu)):
    if counter == 0:
        times.append(float(imu[i][2:]))
    elif counter == 1:
        pitch.append(float(imu[i][1:]))
    elif counter == 2:
        roll.append(float(imu[i][1:]))
    elif counter == 3:
        yaw.append(float(imu[i]))
    counter += 1
    counter = counter % 4

print("Time(ms)      ", "Pitch          ", "Roll            ", "Yaw      ")
for i in range(len(times)):
    print(times[i], "      ", pitch[i], "        ", roll[i], "        ", yaw[i])

In [ ]:
print(tof)

In [ ]:
import statistics as stat
dist_times = []
dist1 = []
dist2 = []

counter = 0
for i in range(len(tof)):
    if counter == 0:
        dist_times.append(int(tof[i][2:]))
    elif counter == 1:
        dist1.append(int(tof[i][2:]))
    elif counter == 2:
        dist2.append(int(tof[i][2:]))
    counter += 1
    counter = counter % 3

print("Time(ms)    ", "Distance 1(cm)    ", "Distance 2(cm)    ")
for i in range(len(dist_times)):
    print(dist_times[i], "      ", dist1[i], "              ", dist2[i])

In [ ]:
import matplotlib.pyplot as plt
plt.plot(dist1)
plt.plot(dist2)
plt.xlabel('Time')
plt.ylabel('Distance(cm)')
plt.show()

In [ ]:
plt.plot(pitch, label = 'pitch')
plt.plot(roll, label = 'roll')
plt.xlabel('Time')
plt.legend()
plt.show()

## Lab 6 - Motors

### Imports and connect to BLE again for convenience

In [ ]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
import time
import numpy as np

LOG.propagate = False

In [10]:
# Get ArtemisBLEController object
ble = get_ble_controller()

# Connect to the Artemis Device
ble.connect()

2023-03-08 17:15:01,543 | INFO     |: Looking for Artemis Nano Peripheral Device: c0:83:24:6a:9a:3c
2023-03-08 17:15:04,215 | INFO     |: Connected to c0:83:24:6a:9a:3c


In [20]:
# Possible strings for SEND_PWM:
# F + pwm, B + pwm, R + pwm, L + pwm, S000 (pwm must be 3 digits)
ble.send_command(CMD.SEND_PWM, "F100")

In [22]:
ble.send_command(CMD.SEND_PWM, "S000")

2023-03-08 17:21:56,476 | INFO     |: Disconnected from 3E8FB54A-DE75-5568-901D-E91E2BD0E736


In [21]:
ble.send_command(CMD.SEND_PWM, "B070")